# Analysis of TCGA DNA Methylation Data and Clinical Data

## Data Processing

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.preprocessing import LabelEncoder
!pwd

/Users/tanishagupta/GRIPS


In [4]:
#pulls raw KIPAN data from local drive and strips away unecessary information
raw_clin_kipan = pd.read_csv('KIPAN.clin.merged.txt', sep='\t', header=None ).T

raw_clin_kipan.columns = raw_clin_kipan.iloc[0]
raw_clin_kipan = raw_clin_kipan[1:]
#raw_clin_kipan

In [5]:
#finds columns with "barcode" in the name. This is essential for finding the correct IDs to match with the methylation data. 
selected_columns = raw_clin_kipan.columns.to_numpy()[raw_clin_kipan.columns.str.contains('barcode')]
#selected_columns = [ i for i in selected_columns if 'drug_barcode' not in i]
selected_columns = [ i for i in selected_columns if '-' not in i]
#selected_columns

In [6]:
#selects only the data with the barcode titled columns
raw_clin_kipan_barcode = raw_clin_kipan.loc[:,selected_columns]
#raw_clin_kipan_barcode

In [7]:
#sets the new column names
raw_clin_kipan_barcode.columns = [s.split('.')[-1] for s in selected_columns]
#raw_clin_kipan_barcode

In [8]:
#drop the barcode columns that are not correct
raw_clin_kipan_barcode.drop(['bcr_drug_barcode','bcr_radiation_barcode',
                             'biospecimen_barcode_bottom','bcr_followup_barcode',
                             'shipment_portion_bcr_aliquot_barcode','bcr_omf_barcode'], 
                            axis=1, errors='ignore')

,bcr_patient_barcode,bcr_sample_barcode,bcr_aliquot_barcode,bcr_analyte_barcode,bcr_portion_barcode,bcr_slide_barcode
1,tcga-kl-8328,tcga-kl-8328-01a,tcga-kl-8328-01a-11d-2308-01,tcga-kl-8328-01a-11d,tcga-kl-8328-01a-11,tcga-kl-8328-01a-01-bs1
2,tcga-kl-8339,tcga-kl-8339-01a,tcga-kl-8339-01a-11d-2308-01,tcga-kl-8339-01a-11d,tcga-kl-8339-01a-11,tcga-kl-8339-01a-01-bs1
3,tcga-km-8439,tcga-km-8439-01a,tcga-km-8439-01a-11d-2308-01,tcga-km-8439-01a-11d,tcga-km-8439-01a-11,tcga-km-8439-01a-01-ts1
4,tcga-km-8441,tcga-km-8441-01a,tcga-km-8441-01a-11d-2308-01,tcga-km-8441-01a-11d,tcga-km-8441-01a-11,tcga-km-8441-01a-01-ts1
5,tcga-km-8442,tcga-km-8442-01a,tcga-km-8442-01a-11d-2308-01,tcga-km-8442-01a-11d,tcga-km-8442-01a-11,tcga-km-8442-01a-01-ts1
...,...,...,...,...,...,...
937,tcga-y8-a896,tcga-y8-a896-01a,tcga-y8-a896-01a-11d-a35y-01,tcga-y8-a896-01a-11d,tcga-y8-a896-01a-11,tcga-y8-a896-01a-01-ts1
938,tcga-y8-a897,tcga-y8-a897-01a,tcga-y8-a897-01a-11d-a35y-01,tcga-y8-a897-01a-11d,tcga-y8-a897-01a-11,tcga-y8-a897-01a-01-ts1
939,tcga-y8-a8ry,tcga-y8-a8ry-01a,tcga-y8-a8ry-01a-11d-a36w-01,tcga-y8-a8ry-01a-11d,tcga-y8-a8ry-01a-11,tcga-y8-a8ry-01a-01-ts1
940,tcga-y8-a8s0,tcga-y8-a8s0-01a,tcga-y8-a8s0-01a-11d-a36w-01,tcga-y8-a8s0-01a-11d,tcga-y8-a8s0-01a-11,tcga-y8-a8s0-01a-01-ts1


In [9]:
# vital_status: The survival state of the person registered on the protocol.
# days_to_death: Number of days between the date used for index and the date from a person's date of death represented as a calculated number of days.

tcga_id_kipan='patient.samples.sample.portions.portion.analytes.analyte.aliquots.aliquot.bcr_aliquot_barcode'

clin_kipan = raw_clin_kipan.loc[:,[tcga_id_kipan, 'admin.disease_code',
'patient.days_to_death','patient.vital_status', 
'patient.age_at_initial_pathologic_diagnosis', 'patient.gender', 'patient.follow_ups.follow_up.person_neoplasm_cancer_status', 
                                  'patient.stage_event.pathologic_stage']]
                                #, 'patient.karnofsky_performance_score']]

#clin_kipan

In [10]:
#set appropriate index for the data table. Sort indexes
clin_kipan = clin_kipan.set_index(tcga_id_kipan)
clin_kipan.index = clin_kipan.index.str.upper()
clin_kipan.sort_index(inplace=True)

In [11]:
#clin_kipan.columns

In [12]:
clin_kipan = clin_kipan[clin_kipan.index.notnull()]

clin_kipan.index = ['-'.join( s.split('-')[:4] ) for s in clin_kipan.index.tolist() ]
#clin_kipan.index

In [13]:
#rename appropriate columns
clin_kipan.rename(columns = {'patient.days_to_death':'days_to_death', 'patient.vital_status':'vital_status', 'patient.age_at_initial_pathologic_diagnosis':'age_at_initial_pathologic_diagnosis', 'patient.gender':'gender', 'admin.disease_code':'subtype', 'patient.follow_ups.follow_up.person_neoplasm_cancer_status':'metastasis_info', 'patient.stage_event.pathologic_stage':'stage'}, inplace = True)

In [14]:
#convert days to death values to floats
vals = clin_kipan['days_to_death'].astype(np.float32)

#create lts and non-lts variable for survival
#current long term survival is two years
clin_kipan['survival'] = 'non-lts'
clin_kipan.loc[pd.to_numeric(clin_kipan.days_to_death).ge(5*365) | clin_kipan.days_to_death.isna(), 'survival'] = 'lts'
#clin_kipan

In [15]:
#read in raw METHYLATION data
meth_kipan = pd.read_csv('KIPAN.hm450.tsv', delim_whitespace=True, header=[0] ).set_index('Composite_Element_REF').drop(['Gene_Symbol','Chromosome','Genomic_Coordinate'], axis=1 ).T

In [16]:
#sort methylation data by index
meth_kipan.sort_index().head()

Composite_Element_REF,cg00000029,cg00000108,cg00000109,cg00000165,cg00000236,cg00000289,cg00000292,cg00000321,cg00000363,cg00000622,...,cg09560636,cg09560650,cg09560658,cg09560723,cg09560763,cg09560811,cg09560911,cg09560912,cg09560953,cg09560979
TCGA-2K-A9WE-01A-11D-A383-05,0.461441,NaN,NaN,0.143910,0.847165,0.737362,0.716795,0.351877,0.248987,0.012136,...,NaN,NaN,0.929777,NaN,0.821841,0.927202,0.018890,NaN,0.844136,NaN
TCGA-2Z-A9J1-01A-11D-A383-05,0.595894,NaN,NaN,0.080724,0.867306,0.706806,0.217862,0.169408,0.173115,0.010890,...,NaN,NaN,0.684141,NaN,0.287309,0.932596,0.018205,NaN,0.253988,NaN
TCGA-2Z-A9J2-01A-11D-A383-05,0.481305,NaN,NaN,0.437447,0.898927,0.758109,0.868605,0.577745,0.567242,0.012268,...,NaN,NaN,0.934065,NaN,0.393975,0.949127,0.027293,NaN,0.898587,NaN
TCGA-2Z-A9J3-01A-12D-A383-05,0.553850,NaN,NaN,0.064233,0.917291,0.675538,0.543087,0.850444,0.470811,0.012568,...,NaN,NaN,0.939414,NaN,0.198644,0.944607,0.019058,NaN,0.648340,NaN
TCGA-2Z-A9J5-01A-21D-A383-05,0.184349,NaN,NaN,0.126119,0.928018,0.677846,0.850474,0.444735,0.204529,0.012233,...,NaN,NaN,0.926087,NaN,0.589858,0.931093,0.029658,NaN,0.722906,NaN


## Determining overlap between methylation and clinical data 

In [17]:
len( set(meth_kipan.index) )

867

In [18]:
len ( set( [ '-'.join( i.split('-')[:4] ) for i in meth_kipan.index ] ) )

867

In [19]:
meth_kipan.index = [ '-'.join( i.split('-')[:4] ) for i in meth_kipan.index ]
meth_kipan.sort_index(inplace=True)
#meth_kipan

In [20]:
len( set( clin_kipan.index ) & set( meth_kipan.index ) )

660

In [21]:
len( set( clin_kipan.index ) - set( meth_kipan.index ) )

281

In [22]:
len( set( meth_kipan.index ) - set( clin_kipan.index ) )

207

In [23]:
#set( meth_kipan.index ) - set( clin_kipan.index )

In [24]:
#determine the number of type of TCGA ID
from collections import Counter

Counter( [i.split('-')[3] for i in meth_kipan.index] )

Counter({'01A': 656, '11A': 205, '01B': 4, '05A': 2})

In [25]:
Counter( [i.split('-')[3] for i in np.array( set( clin_kipan.index ) & set( meth_kipan.index ) ).tolist()])

Counter({'01A': 656, '01B': 4})

In [26]:
#drop these two because they're not matched anywhere
meth_kipan.drop( ['TCGA-DV-A4W0-05A','TCGA-UZ-A9PS-05A'], axis=0, inplace=True, errors = 'ignore')
#meth_kipan

In [27]:
#create a df for normal
clin_normal = pd.DataFrame( index = meth_kipan.index[meth_kipan.index.str.contains('11A')], columns = clin_kipan.columns )
#clin_normal

In [28]:
#set subtype to normal
clin_normal['subtype'] = 'Normal'
#clin_normal

In [29]:
#set kipan values to RCC for all clin_kipan data
clin_kipan['kipan'] = 'RCC'
#clin_kipan

In [30]:
clin_normal['kipan'] = 'Normal'
clin_normal['survival'] = 'Normal'
#clin_normal

In [31]:
#create a df with all normal and tumor values for clinical data
clin_all = pd.concat( [clin_normal, clin_kipan], axis=0 )
clin_all.sort_index( inplace = True )

In [32]:
#clin_all['kipan']

In [33]:
#rename the columns
clin_all = clin_all[['kipan','subtype', 'days_to_death', 'vital_status','survival',
       'age_at_initial_pathologic_diagnosis', 'gender', 'metastasis_info', 'stage']]
#clin_all

## Merging of Two Datasets: clinical and methylation

In [34]:
#final merged df
df_clin_meth = pd.concat([clin_all, meth_kipan], axis='columns', join='inner').sort_values(['kipan'], axis=0)
#df_clin_meth

In [35]:
#determine number of each subtype, very useful for future calcs
Counter( df_clin_meth['subtype'] )

Counter({'Normal': 205, 'kirp': 275, 'kirc': 319, 'kich': 66})

In [36]:
#transpose merged df for ease of use
df_clin_meth = df_clin_meth.T

In [37]:
#df_clin_meth.head(10)

# Data Table Creation!

In [38]:
df_clin_kich = df_clin_meth.iloc[:, 205:]

In [39]:
df_clin_kich = df_clin_kich.sort_values(['subtype'], axis=1)

In [40]:
df_clin_kich = df_clin_kich.iloc[:, 0:66]

In [41]:
df_clin_kirc = df_clin_meth.iloc[:, 205:]

In [42]:
df_clin_kirc = df_clin_kirc.sort_values(['subtype'], axis=1)

In [43]:
df_clin_kirc = df_clin_kirc.iloc[:, 66:385]

In [44]:
df_clin_kirp = df_clin_meth.iloc[:, 205:]

In [45]:
df_clin_kirp = df_clin_kirp.sort_values(['subtype'], axis=1)

In [46]:
df_clin_kirp = df_clin_kirp.iloc[:, 385:]

# Subtying Stat Analysis:

In [47]:
df_stat_temp = pd.DataFrame()

In [48]:
df_stat_temp['ID'] = df_clin_meth.index.tolist()[9:]

In [49]:
df_sorted = df_clin_meth.sort_values("subtype", axis=1)

In [52]:
from statistics import mean
means_kich = []
for i in range (9, len(df_sorted)):
    means_kich.append(mean(df_sorted.iloc[i, 205:271].tolist()))

In [53]:
means_kirc = []
for i in range (9, len(df_sorted)):
    means_kirc.append(mean(df_sorted.iloc[i, 271:590].tolist()))

In [54]:
means_kirp = []
for i in range (9, len(df_sorted)):
    means_kirp.append(mean(df_sorted.iloc[i, 590:len(df_sorted)].tolist()))

In [55]:
df_stat_temp["KICH Means"] = means_kich

In [56]:
df_stat_temp["KIRC Means"] = means_kirc

In [57]:
df_stat_temp["KIRP Means"] = means_kirp

In [58]:
diff_kich_kirc = []
for i in range (0, len(df_stat_temp)):
    diff_kich_kirc.append(means_kich[i] - means_kirc[i])

NameError: name 'df_stat' is not defined

In [ ]:
diff_kirc_kirp = []
for i in range (0, len(df_stat_temp)):
    diff_kirc_kirp.append(means_kirc[i] - means_kirp[i])

In [ ]:
diff_kirp_kich = []
for i in range (0, len(df_stat_temp)):
    diff_kirp_kich.append(means_kirp[i] - means_kich[i])

In [ ]:
df_stat_temp["CH - RC"] = diff_kich_kirc

In [ ]:
df_stat_temp["RC - RP"] = diff_kirc_kirp

In [ ]:
df_stat_temp["RP - CH"] = diff_kirp_kich

In [ ]:
df_stat_temp.sort_values("CH - RC", axis=0).head(10)

In [ ]:
df_temp = df_stat_temp.sort_values("CH - RC", axis=0).head(183201-20000)

In [ ]:
df_temp = df_temp.tail(20000)

In [ ]:
df_temp = df_temp.head(2000)

In [ ]:
df_temp = df_temp.tail(400)

In [ ]:
df_temp = df_temp.head(200)

In [ ]:
df_temp = df_temp.head(100)

In [ ]:
df_stat_temp.sort_values("RC - RP", axis=0).head(10)

In [ ]:
df_temp = df_stat_temp.sort_values("RC - RP", axis=0).tail(40000)

In [ ]:
df_temp = df_temp.head(3000)

In [ ]:
df_temp = df_temp.head(1500)

In [ ]:
df_temp = df_temp.head(1250)

In [ ]:
df_temp = df_temp.tail(250)

In [ ]:
df_stat_temp.sort_values("RP - CH", axis=0).head(10)

In [ ]:
df_temp = df_stat_temp.sort_values("RP - CH", axis=0)

In [ ]:
df_temp = df_temp.tail(40000)

In [ ]:
df_temp = df_temp.head(10000)

In [ ]:
df_temp = df_temp.head(3000)

In [ ]:
df_temp = df_temp.tail(1000)

In [ ]:
df_temp = df_temp.head(200)

# KIRC-KICH

In [ ]:
import matplotlib.patches as mpatches


x = df_sorted.loc['cg08435683'].tolist()[205:271]
x = [i*100 for i in x]
#print(x)
y = df_sorted.loc['cg08435683'].tolist()[271:590]
y = [i*100 for i in y]
#print(y)
z = df_sorted.loc['cg08435683'].tolist()[590:]
z = [i*100 for i in z]
bins = np.linspace(0, 100, 50)

plt.hist([x, y, z], bins, label=['KICH', 'KIRC' 'KIRP'], color=["#7E2E1F", "#FFA026", "#FF0000"])
plt.title('cg08435683 (SLC23A2)')
#colors = [(0.494, 0.180, 0.122), (1, 0.627, 0.149), (1, 0, 0)]
handle_1 = mpatches.Rectangle((0,0),1,1, color=(0.494, 0.180, 0.122), ec="k")
handle_2 = mpatches.Rectangle((0,0),1,1, color=(1, 0.627, 0.149), ec="k")
handle_3 = mpatches.Rectangle((0,0),1,1, color=(1, 0, 0), ec="k")
handles = [handle_1, handle_2, handle_3]
labels = ['KICH', 'KIRC', 'KIRP']
plt.legend(handles, labels)

plt.show()

In [ ]:
x = df_sorted.loc['cg01020475'].tolist()[205:271]
x = [i*100 for i in x]
#print(x)
y = df_sorted.loc['cg01020475'].tolist()[271:590]
y = [i*100 for i in y]
#print(y)
z = df_sorted.loc['cg01020475'].tolist()[590:]
z = [i*100 for i in z]
bins = np.linspace(0, 100, 50)

plt.hist([x, y, z], bins, label=['KICH', 'KIRC' 'KIRP'], color=["#7E2E1F", "#FFA026", "#FF0000"])
plt.title('cg01020475 (ATP4B)')
#colors = [(0.494, 0.180, 0.122), (1, 0.627, 0.149), (1, 0, 0)]
handle_1 = mpatches.Rectangle((0,0),1,1, color=(0.494, 0.180, 0.122), ec="k")
handle_2 = mpatches.Rectangle((0,0),1,1, color=(1, 0.627, 0.149), ec="k")
handle_3 = mpatches.Rectangle((0,0),1,1, color=(1, 0, 0), ec="k")
handles = [handle_1, handle_2, handle_3]
labels = ['KICH', 'KIRC', 'KIRP']
plt.legend(handles, labels)

plt.show()

# KICH - KIRC

In [ ]:
x = df_sorted.loc['cg01288184'].tolist()[205:271]
x = [i*100 for i in x]
#print(x)
y = df_sorted.loc['cg01288184'].tolist()[271:590]
y = [i*100 for i in y]
#print(y)
z = df_sorted.loc['cg01288184'].tolist()[590:]
z = [i*100 for i in z]
bins = np.linspace(0, 100, 50)

plt.hist([x, y, z], bins, label=['KICH', 'KIRC' 'KIRP'], color=["#7E2E1F", "#FFA026", "#FF0000"])
plt.title('cg01288184 (CABLES1)')
#colors = [(0.494, 0.180, 0.122), (1, 0.627, 0.149), (1, 0, 0)]
handle_1 = mpatches.Rectangle((0,0),1,1, color=(0.494, 0.180, 0.122), ec="k")
handle_2 = mpatches.Rectangle((0,0),1,1, color=(1, 0.627, 0.149), ec="k")
handle_3 = mpatches.Rectangle((0,0),1,1, color=(1, 0, 0), ec="k")
handles = [handle_1, handle_2, handle_3]
labels = ['KICH', 'KIRC', 'KIRP']
plt.legend(handles, labels)

plt.show()

In [ ]:
x = df_sorted.loc['cg09479286'].tolist()[205:271]
x = [i*100 for i in x]
#print(x)
y = df_sorted.loc['cg09479286'].tolist()[271:590]
y = [i*100 for i in y]
#print(y)
z = df_sorted.loc['cg09479286'].tolist()[590:]
z = [i*100 for i in z]
bins = np.linspace(0, 100, 50)

plt.hist([x, y, z], bins, label=['KICH', 'KIRC' 'KIRP'], color=["#7E2E1F", "#FFA026", "#FF0000"])
plt.title('cg09479286 (NOSTRIN)')
#colors = [(0.494, 0.180, 0.122), (1, 0.627, 0.149), (1, 0, 0)]
handle_1 = mpatches.Rectangle((0,0),1,1, color=(0.494, 0.180, 0.122), ec="k")
handle_2 = mpatches.Rectangle((0,0),1,1, color=(1, 0.627, 0.149), ec="k")
handle_3 = mpatches.Rectangle((0,0),1,1, color=(1, 0, 0), ec="k")
handles = [handle_1, handle_2, handle_3]
labels = ['KICH', 'KIRC', 'KIRP']
plt.legend(handles, labels)

plt.show()

# KIRP - KIRC

In [ ]:
x = df_sorted.loc['cg00204802'].tolist()[205:271]
x = [i*100 for i in x]
#print(x)
y = df_sorted.loc['cg00204802'].tolist()[271:590]
y = [i*100 for i in y]
#print(y)
z = df_sorted.loc['cg00204802'].tolist()[590:]
z = [i*100 for i in z]
bins = np.linspace(0, 100, 50)

plt.hist([x, y, z], bins, label=['KICH', 'KIRC' 'KIRP'], color=["#7E2E1F", "#FFA026", "#FF0000"])
plt.title('cg00204802 (ATP11A)')
#colors = [(0.494, 0.180, 0.122), (1, 0.627, 0.149), (1, 0, 0)]
handle_1 = mpatches.Rectangle((0,0),1,1, color=(0.494, 0.180, 0.122), ec="k")
handle_2 = mpatches.Rectangle((0,0),1,1, color=(1, 0.627, 0.149), ec="k")
handle_3 = mpatches.Rectangle((0,0),1,1, color=(1, 0, 0), ec="k")
handles = [handle_1, handle_2, handle_3]
labels = ['KICH', 'KIRC', 'KIRP']
plt.legend(handles, labels)

plt.show()

In [ ]:
x = df_sorted.loc['cg01623261'].tolist()[205:271]
x = [i*100 for i in x]
#print(x)
y = df_sorted.loc['cg01623261'].tolist()[271:590]
y = [i*100 for i in y]
#print(y)
z = df_sorted.loc['cg01623261'].tolist()[590:]
z = [i*100 for i in z]
bins = np.linspace(0, 100, 50)

plt.hist([x, y, z], bins, label=['KICH', 'KIRC' 'KIRP'], color=["#7E2E1F", "#FFA026", "#FF0000"])
plt.title('cg01623261 (BAHCC1)')
#colors = [(0.494, 0.180, 0.122), (1, 0.627, 0.149), (1, 0, 0)]
handle_1 = mpatches.Rectangle((0,0),1,1, color=(0.494, 0.180, 0.122), ec="k")
handle_2 = mpatches.Rectangle((0,0),1,1, color=(1, 0.627, 0.149), ec="k")
handle_3 = mpatches.Rectangle((0,0),1,1, color=(1, 0, 0), ec="k")
handles = [handle_1, handle_2, handle_3]
labels = ['KICH', 'KIRC', 'KIRP']
plt.legend(handles, labels)

plt.show()

# KIRC-KIRP

In [ ]:
x = df_sorted.loc['cg00263677'].tolist()[205:271]
x = [i*100 for i in x]
#print(x)
y = df_sorted.loc['cg00263677'].tolist()[271:590]
y = [i*100 for i in y]
#print(y)
z = df_sorted.loc['cg00263677'].tolist()[590:]
z = [i*100 for i in z]
bins = np.linspace(0, 100, 50)

plt.hist([x, y, z], bins, label=['KICH', 'KIRC' 'KIRP'], color=["#7E2E1F", "#FFA026", "#FF0000"])
plt.title('cg00263677 (CRISPLD2)')
#colors = [(0.494, 0.180, 0.122), (1, 0.627, 0.149), (1, 0, 0)]
handle_1 = mpatches.Rectangle((0,0),1,1, color=(0.494, 0.180, 0.122), ec="k")
handle_2 = mpatches.Rectangle((0,0),1,1, color=(1, 0.627, 0.149), ec="k")
handle_3 = mpatches.Rectangle((0,0),1,1, color=(1, 0, 0), ec="k")
handles = [handle_1, handle_2, handle_3]
labels = ['KICH', 'KIRC', 'KIRP']
plt.legend(handles, labels)

plt.show()

In [ ]:
x = df_sorted.loc['cg04125586'].tolist()[205:271]
x = [i*100 for i in x]
#print(x)
y = df_sorted.loc['cg04125586'].tolist()[271:590]
y = [i*100 for i in y]
#print(y)
z = df_sorted.loc['cg04125586'].tolist()[590:]
z = [i*100 for i in z]
bins = np.linspace(0, 100, 50)

plt.hist([x, y, z], bins, label=['KICH', 'KIRC' 'KIRP'], color=["#7E2E1F", "#FFA026", "#FF0000"])
plt.title('cg04125586 (ELF3)')
#colors = [(0.494, 0.180, 0.122), (1, 0.627, 0.149), (1, 0, 0)]
handle_1 = mpatches.Rectangle((0,0),1,1, color=(0.494, 0.180, 0.122), ec="k")
handle_2 = mpatches.Rectangle((0,0),1,1, color=(1, 0.627, 0.149), ec="k")
handle_3 = mpatches.Rectangle((0,0),1,1, color=(1, 0, 0), ec="k")
handles = [handle_1, handle_2, handle_3]
labels = ['KICH', 'KIRC', 'KIRP']
plt.legend(handles, labels)

plt.show()

# KICH - KIRP

In [ ]:
x = df_sorted.loc['cg02748089'].tolist()[205:271]
x = [i*100 for i in x]
#print(x)
y = df_sorted.loc['cg02748089'].tolist()[271:590]
y = [i*100 for i in y]
#print(y)
z = df_sorted.loc['cg02748089'].tolist()[590:]
z = [i*100 for i in z]
bins = np.linspace(0, 100, 50)

plt.hist([x, y, z], bins, label=['KICH', 'KIRC' 'KIRP'], color=["#7E2E1F", "#FFA026", "#FF0000"])
plt.title('cg02748089 (WWOX)')
#colors = [(0.494, 0.180, 0.122), (1, 0.627, 0.149), (1, 0, 0)]
handle_1 = mpatches.Rectangle((0,0),1,1, color=(0.494, 0.180, 0.122), ec="k")
handle_2 = mpatches.Rectangle((0,0),1,1, color=(1, 0.627, 0.149), ec="k")
handle_3 = mpatches.Rectangle((0,0),1,1, color=(1, 0, 0), ec="k")
handles = [handle_1, handle_2, handle_3]
labels = ['KICH', 'KIRC', 'KIRP']
plt.legend(handles, labels)

plt.show()

In [ ]:
x = df_sorted.loc['cg02772121'].tolist()[205:271]
x = [i*100 for i in x]
#print(x)
y = df_sorted.loc['cg02772121'].tolist()[271:590]
y = [i*100 for i in y]
#print(y)
z = df_sorted.loc['cg02772121'].tolist()[590:]
z = [i*100 for i in z]
bins = np.linspace(0, 100, 50)

plt.hist([x, y, z], bins, label=['KICH', 'KIRC' 'KIRP'], color=["#7E2E1F", "#FFA026", "#FF0000"])
plt.title('cg02772121 (TRIM15)')
#colors = [(0.494, 0.180, 0.122), (1, 0.627, 0.149), (1, 0, 0)]
handle_1 = mpatches.Rectangle((0,0),1,1, color=(0.494, 0.180, 0.122), ec="k")
handle_2 = mpatches.Rectangle((0,0),1,1, color=(1, 0.627, 0.149), ec="k")
handle_3 = mpatches.Rectangle((0,0),1,1, color=(1, 0, 0), ec="k")
handles = [handle_1, handle_2, handle_3]
labels = ['KICH', 'KIRC', 'KIRP']
plt.legend(handles, labels)

plt.show()

# KIRP - KICH

In [ ]:
x = df_sorted.loc['cg02670123'].tolist()[205:271]
x = [i*100 for i in x]
#print(x)
y = df_sorted.loc['cg02670123'].tolist()[271:590]
y = [i*100 for i in y]
#print(y)
z = df_sorted.loc['cg02670123'].tolist()[590:]
z = [i*100 for i in z]
bins = np.linspace(0, 100, 50)

plt.hist([x, y, z], bins, label=['KICH', 'KIRC' 'KIRP'], color=["#7E2E1F", "#FFA026", "#FF0000"])
plt.title('cg02670123 (ITGA6)')
#colors = [(0.494, 0.180, 0.122), (1, 0.627, 0.149), (1, 0, 0)]
handle_1 = mpatches.Rectangle((0,0),1,1, color=(0.494, 0.180, 0.122), ec="k")
handle_2 = mpatches.Rectangle((0,0),1,1, color=(1, 0.627, 0.149), ec="k")
handle_3 = mpatches.Rectangle((0,0),1,1, color=(1, 0, 0), ec="k")
handles = [handle_1, handle_2, handle_3]
labels = ['KICH', 'KIRC', 'KIRP']
plt.legend(handles, labels)

plt.show()

In [ ]:
x = df_sorted.loc['cg08435683'].tolist()[205:271]
x = [i*100 for i in x]
#print(x)
y = df_sorted.loc['cg08435683'].tolist()[271:590]
y = [i*100 for i in y]
#print(y)
z = df_sorted.loc['cg08435683'].tolist()[590:]
z = [i*100 for i in z]
bins = np.linspace(0, 100, 50)

plt.hist([x, y, z], bins, label=['KICH', 'KIRC' 'KIRP'], color=["#7E2E1F", "#FFA026", "#FF0000"])
plt.title('cg08435683 (SLC23A2)')
#colors = [(0.494, 0.180, 0.122), (1, 0.627, 0.149), (1, 0, 0)]
handle_1 = mpatches.Rectangle((0,0),1,1, color=(0.494, 0.180, 0.122), ec="k")
handle_2 = mpatches.Rectangle((0,0),1,1, color=(1, 0.627, 0.149), ec="k")
handle_3 = mpatches.Rectangle((0,0),1,1, color=(1, 0, 0), ec="k")
handles = [handle_1, handle_2, handle_3]
labels = ['KICH', 'KIRC', 'KIRP']
plt.legend(handles, labels)

plt.show()

# DMC Stats

In [ ]:
#this is for normal - tumor
import numpy as np

pos_list = 0
neg_list = 0
for i in range(0, len(df_stat_temp)):
    if(True != np.isnan(df_stat_temp.iloc[i, 4])):
        if(df_stat_temp.iloc[i, 4] > 0):
            pos_list += 1
        else:
            neg_list += 1

In [ ]:
diff_kich_normal = []
for i in range (0, len(df_stat)):
    diff_kich_normal.append(means_kich[i] - means_normal[i])

In [ ]:
diff_kirc_normal = []
for i in range (0, len(df_stat)):
    diff_kirc_normal.append(means_kirc[i] - means_normal[i])

In [ ]:
diff_kirp_normal = []
for i in range (0, len(df_stat)):
    diff_kirp_normal.append(means_kirp[i] - means_normal[i])

In [ ]:
df_stat_temp["CH - N"] = diff_kich_normal

In [ ]:
df_stat_temp["RC - N"] = diff_kirc_normal

In [ ]:
df_stat_temp["RP - N"] = diff_kirp_normal

In [ ]:
#this is for KICH - N
import numpy as np

pos_list = 0
neg_list = 0
for i in range(0, len(df_stat_temp)):
    if(True != np.isnan(df_stat_temp.iloc[i, 11])):
        if(df_stat_temp.iloc[i, 11] > 0):
            pos_list += 1
        else:
            neg_list += 1

In [ ]:
#this is for KIRC - N
import numpy as np

pos_list = 0
neg_list = 0
for i in range(0, len(df_stat_temp)):
    if(True != np.isnan(df_stat_temp.iloc[i, 12])):
        if(df_stat_temp.iloc[i, 12] > 0):
            pos_list += 1
        else:
            neg_list += 1

In [ ]:
#this is for KIRP - N
import numpy as np

pos_list = 0
neg_list = 0
for i in range(0, len(df_stat_temp)):
    if(True != np.isnan(df_stat_temp.iloc[i, 13])):
        if(df_stat_temp.iloc[i, 13] > 0):
            pos_list += 1
        else:
            neg_list += 1

In [ ]:
pos_list

In [ ]:
neg_list

# Log Reg for Subtyping KICH (Tumor Only Dataset):

In [ ]:
df_temp = df_clin_meth.iloc[205:,:]
#pd.concat([df_clin_meth.iloc[0:5,:], df_clin_meth.iloc[205:210,:]])

In [ ]:
listy = df_temp["subtype"].tolist()

In [ ]:
#df_temp["subtype"].tolist()

In [ ]:
labels = []
for i in range(0, len(listy)):
    if(listy[i] == "kich"):
        labels.append(1)
    else:
        labels.append(0)

In [ ]:
X = df_temp.loc[:, ['cg08435683','cg01020475','cg01288184','cg09479286','cg02748089','cg02772121','cg02670123','cg08435683']].dropna(axis=1) # Features
y = labels # Target variable

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
# 5 folds selected
kfold = KFold(n_splits=5, random_state=0, shuffle=True)
model = LogisticRegression(solver='liblinear')
results = cross_val_score(model, X, y, cv=kfold)
print(results)
# Output the accuracy. Calculate the mean and std across all folds. 
print("Accuracy: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100.0))

In [ ]:
from sklearn.metrics import roc_curve,auc
from scipy import interp
tprs = []
aucs = []
mean_fpr = np.linspace(0,1,100)
i = 1
for train,test in kfold.split(X,y):
    prediction = model.fit(X,y).predict_proba(X)
    fpr, tpr, t = roc_curve(y, prediction[:, 1])
    tprs.append(interp(mean_fpr, fpr, tpr))
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    #plt.plot(fpr, tpr, lw=2, alpha=0.3, label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))
    i= i+1

plt.plot([0,1],[0,1],linestyle = '--',lw = 2,color = 'black')
mean_tpr = np.mean(tprs, axis=0)
mean_auc = auc(mean_fpr, mean_tpr)
plt.plot(mean_fpr, mean_tpr, color='blue',
         label=r'Mean ROC (AUC = %0.2f )' % (mean_auc),lw=2, alpha=1)

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('K-Fold Validation for KICH vs. OTHER ROC Curve (tumor only data))')
plt.legend(loc="lower right")
plt.text(0.32,0.7,'More accurate area',fontsize = 12)
plt.text(0.63,0.4,'Less accurate area',fontsize = 12)
plt.show()

In [ ]:
test_size = 0.5
seed = 0

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size,
random_state=seed)

model2 = LogisticRegression(solver='liblinear')
model2.fit(X_train, y_train)

predicted = model2.predict(X_test)
matrix = confusion_matrix(y_test, predicted)
print(matrix)
report = classification_report(y_test, predicted)
print(report)

In [ ]:
#define metrics

y_pred_proba = model2.predict_proba(X_test)[::,1]
fpr2, tpr2, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc2 = metrics.roc_auc_score(y_test, y_pred_proba)

#create ROC curve
plt.plot(fpr,tpr,label="Validation="+str(round(roc_auc, 4)),color="#7E2E1F", linewidth=3, linestyle='dashed')
plt.plot(fpr2,tpr2,label="Test="+str(round(auc2, 4)),color="#7E2E1F", linewidth=3)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc=4)
plt.title("KICH vs. OTHER ROC Curve (tumor only data)")
plt.show()

# Log Reg for Subtyping KIRC (Tumor Only Dataset):

In [ ]:
df_temp = df_clin_meth.iloc[205:,:]
#pd.concat([df_clin_meth.iloc[0:5,:], df_clin_meth.iloc[205:210,:]])

In [ ]:
listy = df_temp["subtype"].tolist()

In [ ]:
labels = []
for i in range(0, len(listy)):
    if(listy[i] == "kirc"):
        labels.append(1)
    else:
        labels.append(0)

In [ ]:
X = df_temp.loc[:, ['cg08435683', 'cg01020475', 'cg01288184', 'cg09479286','cg00204802','cg01623261','cg00263677', 'cg04125586']].dropna(axis=1) # Features
y = labels # Target variable

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
# 5 folds selected
kfold = KFold(n_splits=5, random_state=0, shuffle=True)
model = LogisticRegression(solver='liblinear')
results = cross_val_score(model, X, y, cv=kfold)
print(results)
# Output the accuracy. Calculate the mean and std across all folds. 
print("Accuracy: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100.0))

In [ ]:
from sklearn.metrics import roc_curve,auc
from scipy import interp
tprs = []
aucs = []
mean_fpr = np.linspace(0,1,100)
i = 1
for train,test in kfold.split(X,y):
    prediction = model.fit(X,y).predict_proba(X)
    fpr, tpr, t = roc_curve(y, prediction[:, 1])
    tprs.append(interp(mean_fpr, fpr, tpr))
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    #plt.plot(fpr, tpr, lw=2, alpha=0.3, label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))
    i= i+1

plt.plot([0,1],[0,1],linestyle = '--',lw = 2,color = 'black')
mean_tpr = np.mean(tprs, axis=0)
mean_auc = auc(mean_fpr, mean_tpr)
plt.plot(mean_fpr, mean_tpr, color='blue',
         label=r'Mean ROC (AUC = %0.2f )' % (mean_auc),lw=2, alpha=1)

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('K-Fold Validation for KIRC vs. OTHER ROC Curve (tumor only data))')
plt.legend(loc="lower right")
plt.text(0.32,0.7,'More accurate area',fontsize = 12)
plt.text(0.63,0.4,'Less accurate area',fontsize = 12)
plt.show()

In [ ]:
test_size = 0.5
seed = 0

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size,
random_state=seed)

model2 = LogisticRegression(solver='liblinear')
model2.fit(X_train, y_train)

predicted = model2.predict(X_test)
matrix = confusion_matrix(y_test, predicted)
print(matrix)
report = classification_report(y_test, predicted)
print(report)

In [ ]:
#transform confusion matrix into array
#the matrix is stored in a vaiable called confmtrx
confmtrx = np.array(matrix)
#Create DataFrame from confmtrx array 
#rows for test: Churn, No_Churn designation as index 
#columns for preds: Pred_Churn, Pred_NoChurn as column
pd.DataFrame(confmtrx, index=['Normal','KIRC'],
columns=['Predicted_Normal', 'Predicted_KIRC', ])

In [ ]:
y_pred_proba = model2.predict_proba(X_test)[::,1]
fpr2, tpr2, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc2 = metrics.roc_auc_score(y_test, y_pred_proba)

#create ROC curve
plt.plot(fpr,tpr,label="Validation="+str(round(roc_auc, 4)),color="#FF0000", linewidth=3, linestyle='dashed')
plt.plot(fpr2,tpr2,label="Test="+str(round(auc2, 4)),color="#FF0000", linewidth=3)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc=4)
plt.title("KIRC vs. OTHER ROC Curve (tumor only data)")
plt.show()

# Log Reg for Subtyping KIRP (Tumor Only Dataset):

In [ ]:
df_temp = df_clin_meth.iloc[205:,:]
#pd.concat([df_clin_meth.iloc[0:5,:], df_clin_meth.iloc[205:210,:]])

In [ ]:
listy = df_temp["subtype"].tolist()

In [ ]:
labels = []
for i in range(0, len(listy)):
    if(listy[i] == "kirp"):
        labels.append(1)
    else:
        labels.append(0)

In [ ]:
X = df_temp.loc[:, ['cg00204802', 'cg01623261', 'cg00263677', 'cg04125586','cg02748089','cg02772121','cg02670123', 'cg08435683']].dropna(axis=1) # Features
y = labels # Target variable

In [ ]:
# 5 folds selected
kfold = KFold(n_splits=5, random_state=0, shuffle=True)
model = LogisticRegression(solver='liblinear')
results = cross_val_score(model, X, y, cv=kfold)
print(results)
# Output the accuracy. Calculate the mean and std across all folds. 
print("Accuracy: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100.0))

In [ ]:
from sklearn.metrics import roc_curve,auc
from scipy import interp
tprs = []
aucs = []
mean_fpr = np.linspace(0,1,100)
i = 1
for train,test in kfold.split(X,y):
    prediction = model.fit(X,y).predict_proba(X)
    fpr, tpr, t = roc_curve(y, prediction[:, 1])
    tprs.append(interp(mean_fpr, fpr, tpr))
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    #plt.plot(fpr, tpr, lw=2, alpha=0.3, label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))
    i= i+1

plt.plot([0,1],[0,1],linestyle = '--',lw = 2,color = 'black')
mean_tpr = np.mean(tprs, axis=0)
mean_auc = auc(mean_fpr, mean_tpr)
plt.plot(mean_fpr, mean_tpr, color='blue',
         label=r'Mean ROC (AUC = %0.2f )' % (mean_auc),lw=2, alpha=1)

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('K-Fold Validation for KIRC vs. OTHER ROC Curve (including normal data))')
plt.legend(loc="lower right")
plt.text(0.32,0.7,'More accurate area',fontsize = 12)
plt.text(0.63,0.4,'Less accurate area',fontsize = 12)
plt.show()

In [ ]:
test_size = 0.5
seed = 0

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size,
random_state=seed)

model2 = LogisticRegression(solver='liblinear')
model2.fit(X_train, y_train)

predicted = model2.predict(X_test)
matrix = confusion_matrix(y_test, predicted)
print(matrix)
report = classification_report(y_test, predicted)
print(report)

In [ ]:
y_pred_proba = model2.predict_proba(X_test)[::,1]
fpr2, tpr2, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc2 = metrics.roc_auc_score(y_test, y_pred_proba)

#create ROC curve
plt.plot(fpr,tpr,label="Validation="+str(round(roc_auc, 4)),color="#7E2E1F", linewidth=3, linestyle='dashed')
plt.plot(fpr2,tpr2,label="Test="+str(round(auc2, 4)),color="#7E2E1F", linewidth=3)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc=4)
plt.title("KIRP vs. OTHER ROC Curve (tumor only data)")
plt.show()

In [ ]:
#df_clin_meth.drop[['cg0876543', 'cg9056743']]

# KICH or Not? (Using all the data, real world simulation):

In [ ]:
df_temp = df_clin_meth

In [ ]:
listy = df_temp["subtype"].tolist()

In [ ]:
labels = []
for i in range(0, len(listy)):
    if(listy[i] == "kich"):
        labels.append(1)
    else:
        labels.append(0)

In [ ]:
X = df_temp.loc[:, ['cg06607764', 'cg00713400', 'cg07355189','cg06330323','cg00716257','cg07600533','cg00849267','cg04972436','cg06474225','cg02578087','cg02952295','cg01890836','cg05343811','cg07093324','cg03063658','cg02326386',]].dropna(axis=1) # Features
y = labels # Target variable

In [ ]:
# 5 folds selected
kfold = KFold(n_splits=5, random_state=0, shuffle=True)
model = LogisticRegression(solver='liblinear')
results = cross_val_score(model, X, y, cv=kfold)
print(results)
# Output the accuracy. Calculate the mean and std across all folds. 
print("Accuracy: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100.0))

In [ ]:
from sklearn.metrics import roc_curve,auc
from scipy import interp
tprs = []
aucs = []
mean_fpr = np.linspace(0,1,100)
i = 1
for train,test in kfold.split(X,y):
    prediction = model.fit(X,y).predict_proba(X)
    fpr, tpr, t = roc_curve(y, prediction[:, 1])
    tprs.append(interp(mean_fpr, fpr, tpr))
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    #plt.plot(fpr, tpr, lw=2, alpha=0.3, label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))
    i= i+1

plt.plot([0,1],[0,1],linestyle = '--',lw = 2,color = 'black')
mean_tpr = np.mean(tprs, axis=0)
mean_auc = auc(mean_fpr, mean_tpr)
plt.plot(mean_fpr, mean_tpr, color='blue',
         label=r'Mean ROC (AUC = %0.2f )' % (mean_auc),lw=2, alpha=1)

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('K-Fold Validation for KIRC vs. OTHER ROC Curve (including normal data))')
plt.legend(loc="lower right")
plt.text(0.32,0.7,'More accurate area',fontsize = 12)
plt.text(0.63,0.4,'Less accurate area',fontsize = 12)
plt.show()

In [ ]:
test_size = 0.5
seed = 0

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size,
random_state=seed)

model2 = LogisticRegression(solver='liblinear')
model2.fit(X_train, y_train)

predicted = model2.predict(X_test)
matrix = confusion_matrix(y_test, predicted)
print(matrix)
report = classification_report(y_test, predicted)
print(report)

In [ ]:
y_pred_proba = model2.predict_proba(X_test)[::,1]
fpr2, tpr2, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc2 = metrics.roc_auc_score(y_test, y_pred_proba)

#create ROC curve
plt.plot(fpr,tpr,label="Validation="+str(round(roc_auc, 4)),color="#FFA026", linewidth=3, linestyle='dashed')
plt.plot(fpr2,tpr2,label="Test="+str(round(auc2, 4)),color="#FFA026", linewidth=3)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc=4)
plt.title("KICH vs. OTHER ROC Curve (including normal data)")
plt.show()

# KIRC or Not? (Using all the data, real world simulation):

In [ ]:
df_temp = df_clin_meth

In [ ]:
listy = df_temp["subtype"].tolist()

In [ ]:
labels = []
for i in range(0, len(listy)):
    if(listy[i] == "kirc"):
        labels.append(1)
    else:
        labels.append(0)

In [ ]:
X = df_temp.loc[:, ['cg06607764', 'cg00713400', 'cg07355189','cg06330323','cg00716257','cg07600533','cg00849267','cg04972436','cg06474225','cg02578087','cg02952295','cg01890836','cg05343811','cg07093324','cg03063658','cg02326386',]].dropna(axis=1) # Features
y = labels # Target variable

In [ ]:
# 5 folds selected
kfold = KFold(n_splits=5, random_state=0, shuffle=True)
model = LogisticRegression(solver='liblinear')
results = cross_val_score(model, X, y, cv=kfold)
print(results)
# Output the accuracy. Calculate the mean and std across all folds. 
print("Accuracy: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100.0))

In [ ]:
from sklearn.metrics import roc_curve,auc
from scipy import interp
tprs = []
aucs = []
mean_fpr = np.linspace(0,1,100)
i = 1
for train,test in kfold.split(X,y):
    prediction = model.fit(X,y).predict_proba(X)
    fpr, tpr, t = roc_curve(y, prediction[:, 1])
    tprs.append(interp(mean_fpr, fpr, tpr))
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    #plt.plot(fpr, tpr, lw=2, alpha=0.3, label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))
    i= i+1

plt.plot([0,1],[0,1],linestyle = '--',lw = 2,color = 'black')
mean_tpr = np.mean(tprs, axis=0)
mean_auc = auc(mean_fpr, mean_tpr)
plt.plot(mean_fpr, mean_tpr, color='blue',
         label=r'Mean ROC (AUC = %0.2f )' % (mean_auc),lw=2, alpha=1)

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('K-Fold Validation for KIRC vs. OTHER ROC Curve (including normal data))')
plt.legend(loc="lower right")
plt.text(0.32,0.7,'More accurate area',fontsize = 12)
plt.text(0.63,0.4,'Less accurate area',fontsize = 12)
plt.show()

In [ ]:
test_size = 0.5
seed = 0

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size,
random_state=seed)

model2 = LogisticRegression(solver='liblinear')
model2.fit(X_train, y_train)

predicted = model2.predict(X_test)
matrix = confusion_matrix(y_test, predicted)
print(matrix)
report = classification_report(y_test, predicted)
print(report)

In [ ]:
y_pred_proba = model2.predict_proba(X_test)[::,1]
fpr2, tpr2, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc2 = metrics.roc_auc_score(y_test, y_pred_proba)

#create ROC curve
plt.plot(fpr,tpr,label="Validation="+str(round(roc_auc, 4)),color="#FF0000", linewidth=3, linestyle='dashed')
plt.plot(fpr2,tpr2,label="Test="+str(round(auc2, 4)),color="#FF0000", linewidth=3)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc=4)
plt.title("KIRC vs. OTHER ROC Curve (including normal data)")
plt.show()

# KIRP or Not? (Using all the data, real world simulation):

In [ ]:
df_temp = df_clin_meth.dropna(axis=1)

In [ ]:
listy = df_temp["subtype"].tolist()

In [ ]:
labels = []
for i in range(0, len(listy)):
    if(listy[i] == "kirp"):
        labels.append(1)
    else:
        labels.append(0)

In [ ]:
df_temp["subtype_encoded"] = labels

In [ ]:
X = df_temp.loc[:, ['cg06607764', 'cg00713400', 'cg07355189','cg06330323','cg00716257','cg07600533','cg00849267','cg04972436','cg06474225','cg02578087','cg02952295','cg01890836','cg05343811','cg07093324','cg03063658','cg02326386',]].dropna(axis=1) # Features
y = labels # Target variable

In [ ]:
# 5 folds selected
kfold = KFold(n_splits=5, random_state=0, shuffle=True)
model = LogisticRegression(solver='liblinear')
results = cross_val_score(model, X, y, cv=kfold)
print(results)
# Output the accuracy. Calculate the mean and std across all folds. 
print("Accuracy: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100.0))

In [ ]:
from sklearn.metrics import roc_curve,auc
from scipy import interp
tprs = []
aucs = []
mean_fpr = np.linspace(0,1,100)
i = 1
for train,test in kfold.split(X,y):
    prediction = model.fit(X,y).predict_proba(X)
    fpr, tpr, t = roc_curve(y, prediction[:, 1])
    tprs.append(interp(mean_fpr, fpr, tpr))
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    #plt.plot(fpr, tpr, lw=2, alpha=0.3, label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))
    i= i+1

plt.plot([0,1],[0,1],linestyle = '--',lw = 2,color = 'black')
mean_tpr = np.mean(tprs, axis=0)
mean_auc = auc(mean_fpr, mean_tpr)
plt.plot(mean_fpr, mean_tpr, color='blue',
         label=r'Mean ROC (AUC = %0.2f )' % (mean_auc),lw=2, alpha=1)

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('K-Fold Validation for LTS vs. STS (2yr)')
plt.legend(loc="lower right")
plt.text(0.32,0.7,'More accurate area',fontsize = 12)
plt.text(0.63,0.4,'Less accurate area',fontsize = 12)
plt.show()

#change the legend to match

In [ ]:
test_size = 0.5
seed = 0

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size,
random_state=seed)

model2 = LogisticRegression(solver='liblinear')
model2.fit(X_train, y_train)

predicted = model2.predict(X_test)
matrix = confusion_matrix(y_test, predicted)
print(matrix)
report = classification_report(y_test, predicted)
print(report)

In [ ]:
#transform confusion matrix into array
#the matrix is stored in a vaiable called confmtrx
confmtrx = np.array(matrix)
#Create DataFrame from confmtrx array 
#rows for test: Churn, No_Churn designation as index 
#columns for preds: Pred_Churn, Pred_NoChurn as column
pd.DataFrame(confmtrx, index=['Not_KIRP','KIRP'],
columns=['Predicted_Not_KIRP', 'Predicted_KIRP', ])

In [ ]:
#define metrics


#define metrics

y_pred_proba = model2.predict_proba(X_test)[::,1]
fpr2, tpr2, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc2 = metrics.roc_auc_score(y_test, y_pred_proba)

#create ROC curve
plt.plot(fpr,tpr,label="Validation="+str(round(roc_auc, 4)),color="#7E2E1F", linewidth=3, linestyle='dashed')
plt.plot(fpr2,tpr2,label="Test="+str(round(auc2, 4)),color="#7E2E1F", linewidth=3)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc=4)
plt.title("KIRP vs. OTHER ROC Curve (including normal data)")
plt.show()

In [ ]:
#df_temp.head(10)

# normal and kich LR

In [ ]:
df_temp = df_clin_meth.sort_values(by=['subtype']).iloc[0:271, :].dropna(axis=1)

In [ ]:
listy = df_temp["subtype"].tolist()

In [ ]:
labels = []
for i in range(0, len(listy)):
    if(listy[i] == "kich"):
        labels.append(1)
    else:
        labels.append(0)

In [ ]:
df_temp["subtype_encoded"] = labels

In [ ]:
#do i have to find special diff methylated...if i do i will actually die on a hill bro
X = df_temp.loc[:, ['cg06607764', 'cg00713400', 'cg07355189','cg06330323','cg00716257','cg07600533',]].dropna(axis=1) # Features
y = labels # Target variable

In [ ]:
# 5 folds selected
kfold = KFold(n_splits=5, random_state=0, shuffle=True)
model = LogisticRegression(solver='liblinear')
results = cross_val_score(model, X, y, cv=kfold)
print(results)
# Output the accuracy. Calculate the mean and std across all folds. 
print("Accuracy: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100.0))

In [ ]:
from sklearn.metrics import roc_curve,auc
from scipy import interp
tprs = []
aucs = []
mean_fpr = np.linspace(0,1,100)
i = 1
for train,test in kfold.split(X,y):
    prediction = model.fit(X,y).predict_proba(X)
    fpr, tpr, t = roc_curve(y, prediction[:, 1])
    tprs.append(interp(mean_fpr, fpr, tpr))
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    #plt.plot(fpr, tpr, lw=2, alpha=0.3, label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))
    i= i+1

plt.plot([0,1],[0,1],linestyle = '--',lw = 2,color = 'black')
mean_tpr = np.mean(tprs, axis=0)
mean_auc = auc(mean_fpr, mean_tpr)
plt.plot(mean_fpr, mean_tpr, color='blue',
         label=r'Mean ROC (AUC = %0.2f )' % (mean_auc),lw=2, alpha=1)

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('K-Fold Validation for Normal and KICH LR')
plt.legend(loc="lower right")
plt.text(0.32,0.7,'More accurate area',fontsize = 12)
plt.text(0.63,0.4,'Less accurate area',fontsize = 12)
plt.show()

#change the legend to match

In [ ]:
test_size = 0.8
seed = 0

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size,
random_state=seed)

model2 = LogisticRegression(solver='liblinear')
model2.fit(X_train, y_train)

predicted = model2.predict(X_test)
matrix = confusion_matrix(y_test, predicted)
print(matrix)
report = classification_report(y_test, predicted)
print(report)

In [ ]:
#define metrics

y_pred_proba = model2.predict_proba(X_test)[::,1]
fpr2, tpr2, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc2 = metrics.roc_auc_score(y_test, y_pred_proba)

#create ROC curve
plt.plot(fpr,tpr,label="Validation="+str(round(roc_auc, 4)),color="#238C3F", linewidth=3, linestyle='dashed')
plt.plot(fpr2,tpr2,label="Test="+str(round(auc2, 4)),color="#3D03FC", linewidth=3)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc=4)
plt.title("Normal and KICH LR seperation (using only Normal and KICH)")
plt.show()

In [ ]:
df_temp = df_clin_meth.sort_values(by=['subtype'])

In [ ]:
df_temp_1 = df_temp.iloc[0:205, :]

In [ ]:
df_temp_2 = df_temp.iloc[271:590, :]

In [ ]:
df_temp = pd.concat([df_temp_1, df_temp_2], axis=0).dropna(axis=1)
df_temp.index

In [ ]:
listy = df_temp["subtype"].tolist()

In [ ]:
labels = []
for i in range(0, len(listy)):
    if(listy[i] == "kirc"):
        labels.append(1)
    else:
        labels.append(0)

In [ ]:
df_temp["subtype_encoded"] = labels

In [ ]:
#do i have to find special diff methylated...if i do i will actually die on a hill bro
X = df_temp.loc[:, ['cg07355189']].dropna(axis=1) # Features
y = labels # Target variable

In [ ]:
# 5 folds selected
kfold = KFold(n_splits=5, random_state=0, shuffle=True)
model = LogisticRegression(solver='liblinear')
results = cross_val_score(model, X, y, cv=kfold)
print(results)
# Output the accuracy. Calculate the mean and std across all folds. 
print("Accuracy: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100.0))

In [ ]:
from sklearn.metrics import roc_curve,auc
from scipy import interp
tprs = []
aucs = []
mean_fpr = np.linspace(0,1,100)
i = 1
for train,test in kfold.split(X,y):
    prediction = model.fit(X,y).predict_proba(X)
    fpr, tpr, t = roc_curve(y, prediction[:, 1])
    tprs.append(interp(mean_fpr, fpr, tpr))
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    #plt.plot(fpr, tpr, lw=2, alpha=0.3, label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))
    i= i+1

plt.plot([0,1],[0,1],linestyle = '--',lw = 2,color = 'black')
mean_tpr = np.mean(tprs, axis=0)
mean_auc = auc(mean_fpr, mean_tpr)
plt.plot(mean_fpr, mean_tpr, color='blue',
         label=r'Mean ROC (AUC = %0.2f )' % (mean_auc),lw=2, alpha=1)

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('K-Fold Validation for Normal and KIRC LR')
plt.legend(loc="lower right")
plt.text(0.32,0.7,'More accurate area',fontsize = 12)
plt.text(0.63,0.4,'Less accurate area',fontsize = 12)
plt.show()

#change the legend to match

In [ ]:
test_size = 0.8
seed = 0

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size,
random_state=seed)

model2 = LogisticRegression(solver='liblinear')
model2.fit(X_train, y_train)

predicted = model2.predict(X_test)
matrix = confusion_matrix(y_test, predicted)
print(matrix)
report = classification_report(y_test, predicted)
print(report)

In [ ]:
#define metrics

y_pred_proba = model2.predict_proba(X_test)[::,1]
fpr2, tpr2, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc2 = metrics.roc_auc_score(y_test, y_pred_proba)

#create ROC curve
plt.plot(fpr,tpr,label="Validation="+str(round(roc_auc, 4)),color="#238C3F", linewidth=3, linestyle='dashed')
plt.plot(fpr2,tpr2,label="Test="+str(round(auc2, 4)),color="#3D03FC", linewidth=3)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc=4)
plt.title("Normal and KIRC LR seperation (using only Normal and KICH)")
plt.show()

In [ ]:
#biomarkers

In [ ]:
df_clin_meth.head(10)

In [ ]:
df_temp = df_clin_meth.sort_values(by=['subtype'])

In [ ]:
listy = df_temp["subtype"].tolist()

In [ ]:
labels = []
for i in range(0, len(listy)):
    if(listy[i] == "kirc"):
        labels.append(2)
    elif(listy[i] == "kich"):
        labels.append(1)
    elif(listy[i] == "kirp"):
        labels.append(3)
    else:
        labels.append(0)

In [ ]:
df_temp["subtype_encoded"] = labels

In [ ]:
#do i have to find special diff methylated...if i do i will actually die on a hill bro
X = df_temp.loc[:, ['cg06607764', 'cg00713400', 'cg07355189','cg06330323','cg00716257','cg07600533','cg00849267','cg04972436','cg06474225','cg02578087','cg02952295','cg01890836','cg05343811','cg07093324','cg03063658','cg02326386',]].dropna(axis=1) # Features
y = labels # Target variable

In [ ]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
# define the multinomial logistic regression model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
# define the model evaluation procedure
cv = KFold(n_splits=5, random_state=0, shuffle=True)
# evaluate the model and collect the scores
n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print(n_scores)
# report the model performance
print('Mean Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

In [ ]:
# from sklearn.metrics import roc_curve,auc
# from scipy import interp
# tprs = []
# aucs = []
# mean_fpr = np.linspace(0,1,100)
# i = 1
# for train,test in kfold.split(X,y):
#     prediction = model.fit(X,y).predict_proba(X)
#     fpr, tpr, t = roc_curve(y, prediction[:, 1])
#     tprs.append(interp(mean_fpr, fpr, tpr))
#     roc_auc = auc(fpr, tpr)
#     aucs.append(roc_auc)
#     #plt.plot(fpr, tpr, lw=2, alpha=0.3, label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))
#     i= i+1

# plt.plot([0,1],[0,1],linestyle = '--',lw = 2,color = 'black')
# mean_tpr = np.mean(tprs, axis=0)
# mean_auc = auc(mean_fpr, mean_tpr)
# plt.plot(mean_fpr, mean_tpr, color='blue',
#          label=r'Mean ROC (AUC = %0.2f )' % (mean_auc),lw=2, alpha=1)

# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('K-Fold Validation for Normal and KIRC LR')
# plt.legend(loc="lower right")
# plt.text(0.32,0.7,'More accurate area',fontsize = 12)
# plt.text(0.63,0.4,'Less accurate area',fontsize = 12)
# plt.show()

In [ ]:
test_size = 0.8
seed = 0

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size,
random_state=seed)

model2 = LogisticRegression(multi_class='multinomial', solver='lbfgs')
model2.fit(X_train, y_train)

predicted = model2.predict(X_test)
matrix = confusion_matrix(y_test, predicted)
print(matrix)
report = classification_report(y_test, predicted)
print(report)

In [ ]:
# import matplotlib.pyplot as plt
# from sklearn.metrics import RocCurveDisplay

# y_pred_proba = model2.predict_proba(X_test)[::,1]
# #fpr2, tpr2, _ = metrics.roc_curve(y_test,  y_pred_proba)
# #auc2 = metrics.roc_auc_score(y_test, y_pred_proba)

# RocCurveDisplay.from_predictions(
#     y_test,
#     y_pred_proba,
#     name="normal vs the rest",
#     color="darkorange",
# )
# plt.plot([0, 1], [0, 1], "k--", label="chance level (AUC = 0.5)")
# plt.axis("square")
# plt.xlabel("False Positive Rate")
# plt.ylabel("True Positive Rate")
# plt.title("One-vs-Rest ROC curves:\nVirginica vs (Setosa & Versicolor)")
# plt.legend()
# plt.show()